# Examples

This notebooks holds the examples from the paper `Taming the Panda with Python`.

In [3]:
# Panda hostname/IP and Desk login information of your robot
hostname = '192.168.1.11'
username = 'xli8'
password = 'QAZxsw1008'

# panda-py is chatty, activate information log level
import logging
logging.basicConfig(level=logging.INFO)

import panda_py

## Code Block 1

Use the Desk client to connect to the web-application running on the control unit to unlock the brakes and activate the Franka Research Interface (FCI) for robot torque control.

In [4]:
desk = panda_py.Desk(hostname, username, password)
desk.unlock()
desk.activate_fci()

ConnectionError: "BadUsernameOrPassword"

## Code Block 2

Connect to the robot using the Panda class. The default gripper from Franka Emika does not support real-time control and can be controlled using the libfranka bindings directly.

In [3]:
from panda_py import libfranka

panda = panda_py.Panda(hostname)
gripper = libfranka.Gripper(hostname)

INFO:panda:Connected to robot (192.168.1.11).
INFO:panda:Panda class destructor invoked (192.168.1.11).


In [4]:
import time
time1 = time.perf_counter()
q = panda.get_state().q
time2 = time.perf_counter()
print(f"read q time: {time2 - time1}")

time3 = time.perf_counter()
width = gripper.read_once().width
time4 = time.perf_counter()
print(f"read gripper time: {time4 - time3}")

time5 = time.perf_counter()
state = q + [width]
time6 = time.perf_counter()
print(f"concatenate time: {time6 - time5}")



read q time: 0.00011040997924283147
read gripper time: 0.006766395992599428
concatenate time: 5.7668949011713266e-05


In [5]:
panda.get_state().q.append(gripper.read_once().width)

In [9]:
print(panda.get_state().q + [gripper.read_once().width / gripper.read_once().max_width])

[0.8351001022677781, -0.5200546490556573, -0.01984579771753143, -2.5086605354806415, 0.0340050422177213, 1.999580580726794, -0.11812600202660097, 0.9980963212547956]


In [9]:
gripper.read_once().max_width

0.08102806657552719

In [3]:
panda.move_to_start()

INFO:panda:Initializing motion generation (moveToJointPosition).
INFO:motion:Computed joint trajectory: 1 waypoint, duration 3.68 seconds.
INFO:panda:Starting new controller (JointTrajectory).


True

## Code Block 3

Simple motion generation in joint space. The call to `get_pose` produces a $4\times 4$ matrix representing the homogeneous transform from robot base to end-effector. The indices $2,3$ refer to third row and fourth column respectively, i.e. the z-coordinate. The position in z is lowered by $0.1\mathrm{m}$ and passed to the inverse kinematics function to produce joint positions. Finally the call to `move_to_joint_position` generates a motion from the current to the desired joint potions.

In [21]:
panda.move_to_start()
pose = panda.get_pose()
print(pose)
pose[2,3] -= .1
q = panda_py.ik(pose)
print(q)
print(q.__class__)
print(q.shape)
panda.move_to_joint_position(q)

INFO:panda:Stopping active controller (JointTrajectory).
INFO:panda:Initializing motion generation (moveToJointPosition).
INFO:motion:Computed joint trajectory: 1 waypoint, duration 0.80 seconds.
INFO:panda:Starting new controller (JointTrajectory).
INFO:panda:Stopping active controller (JointTrajectory).
INFO:panda:Initializing motion generation (moveToJointPosition).
INFO:motion:Computed joint trajectory: 1 waypoint, duration 0.80 seconds.
INFO:panda:Starting new controller (JointTrajectory).


[[ 9.99986854e-01 -4.61781061e-03  2.22092853e-03  3.07589519e-01]
 [-4.62380405e-03 -9.99985645e-01  2.70109725e-03  2.76962310e-04]
 [ 2.20842349e-03 -2.71133088e-03 -9.99993886e-01  4.86935369e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
[-0.00607587 -0.81522616  0.00422435 -2.6297113   0.00596231  1.81669777
  0.78539816]
<class 'numpy.ndarray'>
(7,)


True

In [5]:
state = panda.get_state()
print(state)
print(state.q)
print(len(state.O_T_EE))
print(panda.get_pose())

{"O_T_EE": [-0.0824112,0.99654,-0.0107613,0,0.996436,0.0825878,0.0171508,0,0.0179802,-0.00930953,-0.999795,0,0.355229,0.00652575,0.458011,1], "O_T_EE_d": [-0.0821546,0.996564,-0.0105725,0,0.996458,0.0823279,0.0171584,0,0.0179698,-0.00912541,-0.999797,0,0.355181,0.00640735,0.457989,1], "F_T_NE": [0.707107,-0.707107,0,0,0.707107,0.707107,0,0,0,0,1,0,0,0,0.1034,1], "NE_T_EE": [1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1], "F_T_EE": [0.707107,-0.707107,0,0,0.707107,0.707107,0,0,0,0,1,0,0,0,0.1034,1], "EE_T_K": [1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1], "m_ee": 0.73, "F_x_Cee": [-0.01,0,0.03], "I_ee": [0.001,0,0,0,0.001,0,0,0,0.001], "m_load": 0, "F_x_Cload": [0,0,0], "I_load": [0,0,0,0,0,0,0,0,0], "m_total": 0.73, "F_x_Ctotal": [-0.01,0,0.03], "I_total": [0.001,0,0,0,0.001,0,0,0,0.001], "elbow": [0.0196287,-1], "elbow_d": [0.0193863,-1], "elbow_c": [0.0193863,-1], "delbow_c": [0,0], "ddelbow_c": [0,0], "tau_J": [0.185707,-9.60369,-0.812317,23.7999,0.70452,2.3847,-0.128491], "tau_J_d": [0,0,0,0,0,0,0], "dtau_J

## Code Block 4

Simple motion generation in Cartesian space. The z-coordinate of the current end-effector pose is lowered by $0.1\mathrm{m}$ as in Code Block 3. However the resulting pose is passed directly to `move_to_pose` to produce a motion in Cartesian space.

In [9]:
panda.move_to_start()
pose = panda.get_pose()
pose[2,3] += .05
# panda.move_to_pose(pose)

INFO:panda:Initializing motion generation (moveToJointPosition).
INFO:motion:Computed joint trajectory: 1 waypoint, duration 5.55 seconds.
INFO:panda:Starting new controller (JointTrajectory).


## Inline Examples

Various short examples used inline.

In [6]:
import time
print(panda.get_state())
print(panda.get_state().q)
print(panda.get_model())
gripper.grasp(0.03, 0.2, 10, 0.04, 0.04)
time1 = time.perf_counter()
print("start")
# gripper.move(1, 0.2)
time2 = time.perf_counter()
print("time: ", time2 - time1)
g = gripper.read_once()
# print(g)
print("width: ", g.width)
# print(g.temperature)
# print(g.time)
# print(g.is_grasped)
print("max width: ", g.max_width)

{"O_T_EE": [0.991808,0.0604815,0.112507,0,0.0554819,-0.99735,0.0470538,0,0.115055,-0.0404262,-0.992536,0,0.337018,0.0424118,0.526428,1], "O_T_EE_d": [0.991803,0.0604794,0.112556,0,0.0554772,-0.99735,0.0470584,0,0.115104,-0.0404283,-0.99253,0,0.337012,0.042412,0.526442,1], "F_T_NE": [0.707107,-0.707107,0,0,0.707107,0.707107,0,0,0,0,1,0,0,0,0.1034,1], "NE_T_EE": [1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1], "F_T_EE": [0.707107,-0.707107,0,0,0.707107,0.707107,0,0,0,0,1,0,0,0,0.1034,1], "EE_T_K": [1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1], "m_ee": 0.73, "F_x_Cee": [-0.01,0,0.03], "I_ee": [0.001,0,0,0,0.001,0,0,0,0.001], "m_load": 0, "F_x_Cload": [0,0,0], "I_load": [0,0,0,0,0,0,0,0,0], "m_total": 0.73, "F_x_Ctotal": [-0.01,0,0.03], "I_total": [0.001,0,0,0,0.001,0,0,0,0.001], "elbow": [0.107617,-1], "elbow_d": [0.107618,-1], "elbow_c": [0.107618,-1], "delbow_c": [0,0], "ddelbow_c": [0,0], "tau_J": [0.058754,-6.04901,-2.49445,23.0461,0.935233,2.41125,0.133349], "tau_J_d": [0,0,0,0,0,0,0], "dtau_J": [-28.24,-178.

In [93]:
max_gripper_width = gripper.read_once().max_width

def convert_gripper_percent_to_width(gripper_percent: float, max_gripper_width: float):
    return float(gripper_percent) * float(max_gripper_width)

import time

gripper_percent = 0.97
inc = 1
while True:
    # if not gripper_queue.empty():
    #     gripper_percent = gripper_queue.get()
    #     width = convert_gripper_percent_to_width(gripper_percent, max_gripper_width)
    #     gripper.move(width=width, speed=0.2)
    #     print(f"desired width: {width}; current width: {gripper.read_once().width}")
    #     gripper_queue.task_done()
    # else:
    #     print(f"no gripper command")

    gripper_percent += 0.001 * inc
    if gripper_percent >= 0.97:
        inc = -1
    if gripper_percent <= 0.03:
        inc = 1
    width = convert_gripper_percent_to_width(gripper_percent, max_gripper_width)
    print("width: ", width)
    time1 = time.perf_counter()
    gripper.move(width=width, speed=1)
    time2 = time.perf_counter()
    print("time: ", time2 - time1)
    print(f"desired width: {width}; current width: {gripper.read_once().width}")

width:  0.07868718003481626
time:  1.1308136279694736
desired width: 0.07868718003481626; current width: 0.07864305377006531
width:  0.07860614277422429
time:  0.8472461750498042
desired width: 0.07860614277422429; current width: 0.07868573814630508
width:  0.0785251055136323
time:  1.255161948967725
desired width: 0.0785251055136323; current width: 0.07868573814630508
width:  0.07844406825304032
time:  1.0753018810646608
desired width: 0.07844406825304032; current width: 0.0785268247127533
width:  0.07836303099244833
time:  0.9312369850231335
desired width: 0.07836303099244833; current width: 0.07848742604255676
width:  0.07828199373185635
time:  0.8832577510038391
desired width: 0.07828199373185635; current width: 0.07841059565544128
width:  0.07820095647126436
time:  0.9152464569779113
desired width: 0.07820095647126436; current width: 0.0783245712518692
width:  0.07811991921067238
time:  0.9352219690335914
desired width: 0.07811991921067238; current width: 0.0782451182603836
width:

KeyboardInterrupt: 

## Code Block 5

Using the integrated logging mechanism, the libfranka `RobotState` can be logged at a frequency of $1\mathrm{kHz}$. This example creates two end-effector poses `T_0` and `T_1` based on the starting pose, displaced $0.25\mathrm{m}$ to the left and right respectively. Before a motion is generated between these poses, logging is enabled for this Panda instance (line 9). The enable logging function takes the buffer size in number of steps as argument. As such, $2e3$ steps at $1\mathrm{kHz}$ correspond to a buffer holding the state of the past 2 seconds. After the motion is finished, logging is disabled and the buffer retrieved (line 14).

In [12]:
from panda_py import constants
import numpy as np

T_0 = panda_py.fk(constants.JOINT_POSITION_START)
T_0[1, 3] = 0.25
T_1 = T_0.copy()
T_1[1, 3] = -0.25

panda.move_to_pose(T_0)
panda.enable_logging(40000)
panda.move_to_pose(T_1,
                   speed_factor=0.01,
                   stiffness=2 * np.array([600, 600, 600, 600, 250, 150, 50]))
panda.disable_logging()
log = panda.get_log()

panda.move_to_pose(T_0)
panda.enable_logging(10000)
panda.move_to_joint_position(panda_py.ik(T_1))
panda.disable_logging()
joint_log = panda.get_log()

INFO:panda:Stopping active controller (CartesianTrajectory).
INFO:panda:Initializing motion generation (moveToPose).
INFO:motion:Computed Cartesian trajectory: 1 waypoint, duration 0.87 seconds.
INFO:panda:Starting new controller (CartesianTrajectory).


TypeError: move_to_pose(): incompatible function arguments. The following argument types are supported:
    1. (self: panda_py._core.Panda, positions: list[numpy.ndarray[numpy.float64[3, 1]]], orientations: list[numpy.ndarray[numpy.float64[4, 1]]], speed_factor: float = 0.2, impedance: numpy.ndarray[numpy.float64[6, 6]] = array([[800., 0., 0., 0., 0., 0.], [ 0., 800., 0., 0., 0., 0.], [ 0., 0., 800., 0., 0., 0.], [ 0., 0., 0., 40., 0., 0.], [ 0., 0., 0., 0., 40., 0.], [ 0., 0., 0., 0., 0., 40.]]), damping_ratio: float = 1.0, nullspace_stiffness: float = 15.0, dq_threshold: float = 0.001, success_threshold: float = 0.01) -> bool
    2. (self: panda_py._core.Panda, position: numpy.ndarray[numpy.float64[3, 1]], orientation: numpy.ndarray[numpy.float64[4, 1]], speed_factor: float = 0.2, impedance: numpy.ndarray[numpy.float64[6, 6]] = array([[800., 0., 0., 0., 0., 0.], [ 0., 800., 0., 0., 0., 0.], [ 0., 0., 800., 0., 0., 0.], [ 0., 0., 0., 40., 0., 0.], [ 0., 0., 0., 0., 40., 0.], [ 0., 0.,

In [8]:
from matplotlib import pyplot as plt

def plot(O_T_EE, ax, title: str):
  ax.plot(O_T_EE[:,13], O_T_EE[:,14])
  ax.set_xlim(-0.3,0.3)
  ax.set_ylim(0.25,0.75)
  ax.set_xlabel('y (m)')
  ax.set_ylabel('z (m)')
  ax.grid(True)
  ax.set_title(title)

fig, (ax0, ax1) = plt.subplots(1, 2, figsize=(6,3))
plot(np.array(log['O_T_EE']), ax0, 'Cartesian Motion Path')
plot(np.array(joint_log['O_T_EE']), ax1, 'Joint Motion Path')
fig.tight_layout()

## Code Block 6

Running a panda-py controller. After initializing the controller, the current position and orientation are stored in `x0` and `q0` respectively, where `q0` is a quaternion representation of the end-effector orientation. After starting the controller, a `PandaContext` is created from the Panda object (line 10). `PandaContext` is a convenient context manager that executes a loop at a fixed frequency for an optional amount of time. The call to `PandaContext.ok` throttles the loop and also raises any control exceptions that may have been raised by libfranka. Use of `PandaContext` is optional and users are free to manage the control flow how they wish. In this example a periodic linear displacement along the y-axis is added to the initial pose (line 13). This results in the end-effector moving periodically from left to right in straight lines.

> **Info**
>
> This Code Block doesn't run well inside IPython notebooks and may not finish successfully. Please run the example located at `examples/cartesian_impedance.py` instead.


## Code Block 7

Code Block 7: Resolved rate controller with reactive manipulability maximization. This example is from the Robotics Toolbox for Python. To run it on the real hardware with panda-py requires only connecting the inputs and outputs of the control loop to panda-py, i.e., using the joint positions `Panda.q` and providing the control signal to `IntegrateVelocity.set control`. Additionally, the inequality constraints to avoid the joint limits were removed, as panda-py controllers already have integrated joint limit avoidance using impedance control.

> **Info**
>
> This Code Block doesn't run well inside IPython notebooks and may not finish successfully. Please run the example located at `examples/mmc.py` instead.